In [1]:
import torch

In [14]:
s[:, 2:4].shape

torch.Size([2, 2, 5, 3])

In [8]:
s = torch.arange(120).reshape(2,4,5,3)

print(s[:,:,2:4])
print(s[:,:,2:4].max(2))

torch.max(s, 1/s)

tensor([[[[  6,   7,   8],
          [  9,  10,  11]],

         [[ 21,  22,  23],
          [ 24,  25,  26]],

         [[ 36,  37,  38],
          [ 39,  40,  41]],

         [[ 51,  52,  53],
          [ 54,  55,  56]]],


        [[[ 66,  67,  68],
          [ 69,  70,  71]],

         [[ 81,  82,  83],
          [ 84,  85,  86]],

         [[ 96,  97,  98],
          [ 99, 100, 101]],

         [[111, 112, 113],
          [114, 115, 116]]]])
torch.return_types.max(
values=tensor([[[  9,  10,  11],
         [ 24,  25,  26],
         [ 39,  40,  41],
         [ 54,  55,  56]],

        [[ 69,  70,  71],
         [ 84,  85,  86],
         [ 99, 100, 101],
         [114, 115, 116]]]),
indices=tensor([[[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],

        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]]]))


tensor([[[[ inf,   1.,   2.],
          [  3.,   4.,   5.],
          [  6.,   7.,   8.],
          [  9.,  10.,  11.],
          [ 12.,  13.,  14.]],

         [[ 15.,  16.,  17.],
          [ 18.,  19.,  20.],
          [ 21.,  22.,  23.],
          [ 24.,  25.,  26.],
          [ 27.,  28.,  29.]],

         [[ 30.,  31.,  32.],
          [ 33.,  34.,  35.],
          [ 36.,  37.,  38.],
          [ 39.,  40.,  41.],
          [ 42.,  43.,  44.]],

         [[ 45.,  46.,  47.],
          [ 48.,  49.,  50.],
          [ 51.,  52.,  53.],
          [ 54.,  55.,  56.],
          [ 57.,  58.,  59.]]],


        [[[ 60.,  61.,  62.],
          [ 63.,  64.,  65.],
          [ 66.,  67.,  68.],
          [ 69.,  70.,  71.],
          [ 72.,  73.,  74.]],

         [[ 75.,  76.,  77.],
          [ 78.,  79.,  80.],
          [ 81.,  82.,  83.],
          [ 84.,  85.,  86.],
          [ 87.,  88.,  89.]],

         [[ 90.,  91.,  92.],
          [ 93.,  94.,  95.],
          [ 96.,  97.,  98

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from copy import deepcopy

In [4]:
s = torch.randn([32, 64, 128, 256])

In [5]:
print(s.T.shape)
print(s.mT.shape)

torch.Size([256, 128, 64, 32])
torch.Size([32, 64, 256, 128])


/tmp/ipykernel_22902/396305090.py:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:2244.)
  print(s.T.shape)


In [15]:
!pip install h5py

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 4.5 MB 6.7 MB/s eta 0:00:01


In [16]:
import pathlib

In [18]:
test = pathlib.Path("./models/")

In [24]:
list(test.glob('**/*'))

[PosixPath('models/resnet.py'),
 PosixPath('models/torch_util.py'),
 PosixPath('models/__pycache__'),
 PosixPath('models/.ipynb_checkpoints'),
 PosixPath('models/__init__.py'),
 PosixPath('models/__pycache__/torch_util.cpython-38.pyc'),
 PosixPath('models/__pycache__/__init__.cpython-38.pyc'),
 PosixPath('models/__pycache__/resnet.cpython-38.pyc')]

In [31]:
tt = [1, 2, 3]
tt[::-1]

[3, 2, 1]

In [2]:
OPS = {
    'none' : lambda C_in, C_out, stride: Zero(stride),
    'conv3' : lambda C_in, C_out, stride: nn.Conv2d(C_in, C_out, 3, stride, padding=1),
    'conv5' : lambda C_in, C_out, stride: nn.Conv2d(C_in, C_out, 5, stride, padding=2)
}

class Zero(nn.Module):

    def __init__(self, stride):
        super(Zero, self).__init__()
        self.stride = stride

    def forward(self, x):
        if self.stride == 1:
            return x.mul(0.)
        return x[:,:,::self.stride,::self.stride].mul(0.)

op_name = list(OPS.keys())
print(op_name)

['none', 'conv3', 'conv5']


In [3]:
class NAS201SearchCell(nn.Module):
    def __init__(
        self,
        C_in,
        C_out,
        stride,
        max_nodes,
        op_names):
        
        super(NAS201SearchCell, self).__init__()

        self.op_names = deepcopy(op_names)
        self.edges = nn.ModuleDict()
        self.max_nodes = max_nodes
        self.in_dim = C_in
        self.out_dim = C_out
        
        for i in range(1, max_nodes):
            for j in range(i):
                node_str = "{:}<-{:}".format(i, j)
                if j == 0:
                    xlists = [
                        OPS[op_name](C_in, C_out, stride)
                        for op_name in op_names
                    ]
                else:
                    xlists = [
                        OPS[op_name](C_in, C_out, 1)
                        for op_name in op_names
                    ]
                self.edges[node_str] = nn.ModuleList(xlists)
                
        self.edge_keys = sorted(list(self.edges.keys()))
        self.edge2index = {key: i for i, key in enumerate(self.edge_keys)}
        self.index2key = {i:key for i, key in enumerate(self.edge_keys)}
        self.num_edges = len(self.edges)
        self.register_buffer('alphas', nn.Parameter(1e-3 * torch.randn(self.num_edges, len(op_names))))
        

    def extra_repr(self):
        string = "info :: {max_nodes} nodes, inC={in_dim}, outC={out_dim}".format(
            **self.__dict__
        )
        return string
    
    def get_arch_parameters(self):
        _arch_parameters = []
        
        
        for k, v in self.named_parameters():
            if k.endswith('alpha'):
                _arch_parameters.append(v)
        
        return _arch_parameters
    

    def forward(self, inputs):
        nodes = [inputs]
        for i in range(1, self.max_nodes):
            inter_nodes = []
            for j in range(i):
                node_str = "{:}<-{:}".format(i, j)
                weights = self.alphas[self.edge2index[node_str]]
                inter_nodes.append(
                    sum(
                        layer(nodes[j]) * w
                        for layer, w in zip(self.edges[node_str], weights)
                    )
                )
            nodes.append(sum(inter_nodes))
        return nodes[-1]


In [4]:
nas_cell = NAS201SearchCell(32, 32, 1, 5, op_name)

In [10]:
nas_cell = nas_cell.cuda()

dummy_input = torch.ones(64, 32, 16, 16).cuda()

out = nas_cell(dummy_input)

In [13]:
out.backward(torch.ones(64,32,16,16).cuda())

In [31]:
max_len = 6
op_names = sorted(OPS.keys())
edges = nn.ModuleDict()
for i in range(1, max_len):
    for j in range(i):
        node_str = "{:}<-{:}".format(i, j)
        if j == 0:
            xlists = [
                OPS[op_name](16, 3)
                for op_name in op_names
            ]
        else:
            xlists = [
                OPS[op_name](16, 3)
                for op_name in op_names
            ]
        edges[node_str]= nn.ModuleList(xlists)

edges_keys = sorted(list(edges.keys()))
edge2index = {key:i for i, key in enumerate(edges_keys)}
num_edge = len(edges_keys)


In [39]:
weights = nn.Parameter(torch.ones([3, 4]))

In [41]:
weights[2]

tensor([1., 1., 1., 1.], grad_fn=<SelectBackward0>)

In [38]:
for i, weight in zip([1,2,3], weights):
    print(i, weight)

1 tensor(1., grad_fn=<UnbindBackward0>)
2 tensor(1., grad_fn=<UnbindBackward0>)
3 tensor(1., grad_fn=<UnbindBackward0>)
